정답 필드를 행복지수(2)로 하고, 건강(입력값은 필드 3~:-1)


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

파일 열기

In [10]:
df=pd.read_excel('../data02/행복지수(2021).xlsx')
df
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   순위          25 non-null     int64  
 1   지역          25 non-null     object 
 2   행복지수        25 non-null     float64
 3   건강상태        25 non-null     float64
 4   재정상태        25 non-null     float64
 5   친지_친구와의 관계  25 non-null     float64
 6   가정생활        25 non-null     float64
 7   사회생활        25 non-null     float64
dtypes: float64(6), int64(1), object(1)
memory usage: 1.7+ KB


In [ ]:
data=df.values[:, 2:].astype('float') #필요한 데이타까지만 다 가지고 옴
print(data)

[[6.53 6.86 5.99 6.65 6.66 6.48]
 [6.45 6.81 5.91 6.55 6.55 6.42]
 [6.38 6.7  5.78 6.54 6.52 6.39]
 [6.38 6.72 5.8  6.48 6.54 6.35]
 [6.38 6.66 5.88 6.51 6.54 6.32]
 [6.36 6.71 5.85 6.44 6.53 6.29]
 [6.35 6.61 5.84 6.5  6.49 6.33]
 [6.33 6.54 5.87 6.45 6.55 6.23]
 [6.32 6.67 5.8  6.43 6.45 6.27]
 [6.29 6.72 5.79 6.38 6.44 6.12]
 [6.29 6.7  5.63 6.47 6.5  6.14]
 [6.28 6.64 5.83 6.32 6.36 6.26]
 [6.27 6.6  5.6  6.4  6.42 6.32]
 [6.24 6.57 5.68 6.31 6.45 6.18]
 [6.24 6.66 5.69 6.29 6.42 6.13]
 [6.23 6.5  5.77 6.31 6.38 6.19]
 [6.23 6.66 5.64 6.35 6.42 6.07]
 [6.19 6.54 5.64 6.27 6.39 6.1 ]
 [6.19 6.55 5.51 6.46 6.4  6.03]
 [6.18 6.46 5.56 6.32 6.43 6.15]
 [6.13 6.62 5.57 6.21 6.27 5.97]
 [6.13 6.52 5.5  6.24 6.38 6.03]
 [6.1  6.51 5.47 6.23 6.28 6.03]
 [6.06 6.43 5.42 6.19 6.3  5.98]
 [6.03 6.39 5.42 6.12 6.31 5.93]]


In [15]:
y_data=torch.from_numpy(data[:,0]).view(-1,1)
print(y_data)
x_data=torch.from_numpy(data[:,1:]) #행은 전부다, 열은 1부터 끝까지
x_data


tensor([[6.5300],
        [6.4500],
        [6.3800],
        [6.3800],
        [6.3800],
        [6.3600],
        [6.3500],
        [6.3300],
        [6.3200],
        [6.2900],
        [6.2900],
        [6.2800],
        [6.2700],
        [6.2400],
        [6.2400],
        [6.2300],
        [6.2300],
        [6.1900],
        [6.1900],
        [6.1800],
        [6.1300],
        [6.1300],
        [6.1000],
        [6.0600],
        [6.0300]], dtype=torch.float64)


tensor([[6.8600, 5.9900, 6.6500, 6.6600, 6.4800],
        [6.8100, 5.9100, 6.5500, 6.5500, 6.4200],
        [6.7000, 5.7800, 6.5400, 6.5200, 6.3900],
        [6.7200, 5.8000, 6.4800, 6.5400, 6.3500],
        [6.6600, 5.8800, 6.5100, 6.5400, 6.3200],
        [6.7100, 5.8500, 6.4400, 6.5300, 6.2900],
        [6.6100, 5.8400, 6.5000, 6.4900, 6.3300],
        [6.5400, 5.8700, 6.4500, 6.5500, 6.2300],
        [6.6700, 5.8000, 6.4300, 6.4500, 6.2700],
        [6.7200, 5.7900, 6.3800, 6.4400, 6.1200],
        [6.7000, 5.6300, 6.4700, 6.5000, 6.1400],
        [6.6400, 5.8300, 6.3200, 6.3600, 6.2600],
        [6.6000, 5.6000, 6.4000, 6.4200, 6.3200],
        [6.5700, 5.6800, 6.3100, 6.4500, 6.1800],
        [6.6600, 5.6900, 6.2900, 6.4200, 6.1300],
        [6.5000, 5.7700, 6.3100, 6.3800, 6.1900],
        [6.6600, 5.6400, 6.3500, 6.4200, 6.0700],
        [6.5400, 5.6400, 6.2700, 6.3900, 6.1000],
        [6.5500, 5.5100, 6.4600, 6.4000, 6.0300],
        [6.4600, 5.5600, 6.3200, 6.4300, 6.1500],


float64니까 32로 내리자.

In [22]:
x_data = x_data.float()
y_data = y_data.float()

In [23]:
print(x_data.dtype, y_data.dtype)
print(model(x_data).shape)

torch.float32 torch.float32
torch.Size([25, 1])


target(y) : 행복지수
입력값(x) : 건강상태, 재정상태, 친지_친구와의 관계, 가정생활, 사회생활
모델: 다중선형회귀 (Linear Regression)

모듈법

model 생성

In [24]:
y_data.shape

torch.Size([25, 1])

In [25]:
model = nn.Linear(5,1)
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.3315, -0.1998,  0.2724,  0.2574,  0.2181]], requires_grad=True), Parameter containing:
tensor([0.0963], requires_grad=True)]


최적화

In [26]:
optimizer=optim.SGD(model.parameters(), lr=0.0004)

In [27]:
print(x_data.shape)   # torch.Size([25,5])
print(y_data.shape)   # torch.Size([25,1])
print(model(x_data).shape)  # torch.Size([25,1])

torch.Size([25, 5])
torch.Size([25, 1])
torch.Size([25, 1])


In [28]:
nb_epochs=20000

for epoch in range(nb_epochs):
  prediction = model(x_data)
  cost=F.mse_loss(prediction, y_data)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%400==0:
    print('Epoch:', epoch, 'Cost:', cost.item())

Epoch: 0 Cost: 0.13725963234901428
Epoch: 400 Cost: 0.001396296895109117
Epoch: 800 Cost: 0.0013893063878640532
Epoch: 1200 Cost: 0.0013823318295180798
Epoch: 1600 Cost: 0.0013753802049905062
Epoch: 2000 Cost: 0.0013684502337127924
Epoch: 2400 Cost: 0.0013615761417895555
Epoch: 2800 Cost: 0.001354727428406477
Epoch: 3200 Cost: 0.0013479277258738875
Epoch: 3600 Cost: 0.001341161783784628
Epoch: 4000 Cost: 0.0013344420585781336
Epoch: 4400 Cost: 0.0013277612160891294
Epoch: 4800 Cost: 0.0013211033074185252
Epoch: 5200 Cost: 0.0013144970871508121
Epoch: 5600 Cost: 0.001307917875237763
Epoch: 6000 Cost: 0.0013014000141993165
Epoch: 6400 Cost: 0.0012948985677212477
Epoch: 6800 Cost: 0.0012884248280897737
Epoch: 7200 Cost: 0.0012819682015106082
Epoch: 7600 Cost: 0.001275561167858541
Epoch: 8000 Cost: 0.0012691807933151722
Epoch: 8400 Cost: 0.001262850477360189
Epoch: 8800 Cost: 0.001256538089364767
Epoch: 9200 Cost: 0.0012502530589699745
Epoch: 9600 Cost: 0.0012439879355952144
Epoch: 10000 C

In [29]:
new_data=torch.FloatTensor([[6.5, 5.8, 6.2, 6.0, 5.9]])
y_pred=model(new_data)
print(y_pred)


tensor([[5.9815]], grad_fn=<AddmmBackward0>)


클래스법

In [ ]:
class MultiLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    #다중선형회귀, input_dim=3, output_dim=1
    self.linear=nn.Linear(3,1)

  def forward(self, x):
    return self.linear(x)   

In [ ]:
model=Mu

데이터 범위 잡기

In [7]:
y = df['행복지수']
y

0     6.53
1     6.45
2     6.38
3     6.38
4     6.38
5     6.36
6     6.35
7     6.33
8     6.32
9     6.29
10    6.29
11    6.28
12    6.27
13    6.24
14    6.24
15    6.23
16    6.23
17    6.19
18    6.19
19    6.18
20    6.13
21    6.13
22    6.10
23    6.06
24    6.03
Name: 행복지수, dtype: float64

In [9]:
x = df[['건강상태', '재정상태', '친지_친구와의 관계', '가정생활', '사회생활']]
x

,건강상태,재정상태,친지_친구와의 관계,가정생활,사회생활
0,6.86,5.99,6.65,6.66,6.48
1,6.81,5.91,6.55,6.55,6.42
2,6.70,5.78,6.54,6.52,6.39
3,6.72,5.80,6.48,6.54,6.35
4,6.66,5.88,6.51,6.54,6.32
5,6.71,5.85,6.44,6.53,6.29
6,6.61,5.84,6.50,6.49,6.33
7,6.54,5.87,6.45,6.55,6.23
8,6.67,5.80,6.43,6.45,6.27
9,6.72,5.79,6.38,6.44,6.12


y는 컬럼 하나만 꺼냈기 때문에 1차원,
x는 2차원이라서 모양이 다르게 보이는거임.

선형회귀는 이렇게 생긴 문제를 원해.

X : (n행, m열)  ← 여러 입력 변수
y : (n행,)     ← 하나의 정답

X 👉 (25, 5) DataFrame ✅
y 👉 (25,) Series ✅